In [14]:

from pathlib import Path
import sys, pandas as pd

# src/
THIS = Path.cwd()
SRC  = (THIS / "./src").resolve()
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

from src.morph_ngram_dp_dict import build_ngram_dict_from_top_hu_words, segment_morphs_enphon, decode_morphs_with_dict, \
    dict_to_rows, save_mapping_csv, load_mapping_csv

OUT_DIR = Path("./data/coverage"); OUT_DIR.mkdir(parents=True, exist_ok=True)
CSV_MAP = OUT_DIR / "morph_ngram_dp_dict.csv"  # the final merged mapping

In [ ]:
mapping = build_ngram_dict_from_top_hu_words(
    test_n_words=60_000,
    cmu_max_words=15_000,
    min_test_count=4,    # “more than 3 times” → keep ≥4
    ins=1.0, dele=1.0
)

save_mapping_csv(mapping, CSV_MAP)
# 2) Inspect/save the table
df = pd.DataFrame(dict_to_rows(mapping)).sort_values(["n","dp_cost","count"])
print(df.head())

   hu_ngram  n  count en_word  en_pron  dp_cost
0        az  1      0      oz      ɑ z      0.0
3        eǧ  1      0    edge     ɛ dʒ      0.0
6      volt  1      0   vault  v ɔ l t      0.0
9        el  1      0       l      ɛ l      0.0
11       ki  1      0     key      k i      0.0


In [16]:
mapping_csv = load_mapping_csv(CSV_MAP)

# 3) Decode a new sentence with tri vs (uni+bi) choice
morphs = segment_morphs_enphon("Tűnj el, tűnj el te vagy az igazi bűnjel. Áradás.")
print(morphs)
words, total_cost = decode_morphs_with_dict(morphs, mapping_csv, max_n=3)
print("morphs:", morphs)
print("EN words:", words, "cost:", total_cost)

['tűƝ', 'el,', 'tűƝ', 'el', 'te', 'vaǧ', 'az', 'igazi', 'bűƝel.', 'ár', 'ad', 'ás.']
morphs: ['tűƝ', 'el,', 'tűƝ', 'el', 'te', 'vaǧ', 'az', 'igazi', 'bűƝel.', 'ár', 'ad', 'ás.']
EN words: [] cost: inf
